<div class="alert alert-info">
<h1><center><font color=darkblue>GIẢI PHƯƠNG TRÌNH $AX = B$ BẰNG GAUSS JORDAN<font></center></h1>


</div>

## <font color=RoyalBlue>Ý tưởng thuật toán:<font>
- B1: Tìm hạng của ma trận sau đó nhận xét.
- B2: Tìm phần tử giải $\alpha_{pq}$
- B3: Biến đổi ma trận theo công thức: giữ nguyên hàng p, với các hàng còn lại thì ${h_i}={h_i}-\frac{\alpha_{iq}}{\alpha_{pq}}$. Khi đó cột thứ q sẽ bằng 0 tại các phẩn tử khác ngoài phần tử trội.
- B4: Thực hiện B2, B3 với các hàng khác
- B5: Chuẩn hóa phần tử giải
- B6: Đưa ra kết luận và in kết quả.

### <font color=RoyalBlue>Import các thư viện cần thiết:<font>
- Numpy để làm việc với mảng
- Pandas để làm việc với file dạng csv hoặc xlsx

In [1]:
import numpy as np
import pandas as pd

### <font color=RoyalBlue>Đọc file matran.xlsx <font>
và đưa về dạng mảng

In [2]:
df = pd.read_excel('matran.xlsx', header = None)
df

,0,1,2
0,0,0,0
1,2,4,8
2,1,2,4
3,0,0,0


In [3]:
matrix = df.to_numpy()
print(matrix)

[[0 0 0]
 [2 4 8]
 [1 2 4]
 [0 0 0]]


### <font color=RoyalBlue>Khởi tạo một số mảng phụ lưu kết quả <font>
- index_row: lưu các hàng của phần tử giải
- index_column: lưu các cột của phần tử giải
- result: ma trận lưu kết quả với các giá trị ban đầu là 0

In [4]:
index_row = []  
index_column = []  
result = np.zeros((len(matrix[0]) - 1, len(matrix[0])))

In [5]:
print(matrix)

[[0 0 0]
 [2 4 8]
 [1 2 4]
 [0 0 0]]


In [6]:
print(result)

[[0. 0. 0.]
 [0. 0. 0.]]


### <font color=RoyalBlue>Khởi tạo hàm tìm phần tử giải <font>
<br>Thuật toán:<br/>
- B1: tạo biến pivot_element (phần tử giải), mảng index_temp (mảng cho phần tử có trị tuyệt đối lớn nhất trong hàng.
- B2: Cho row chạy tới len(matrix) (kích thước của ma trận $A$)
- B3: Kiểm tra nếu hàng này đã có phần tử giải thì bỏ qua
- B4: Tìm phần tử có trị tuyệt đối lớn nhất trong hàng
- B5: Nếu trong hàng có chứ $1$ hoặc $-1$ chuyển sang bước 6, trái lại chuyển sang bước 9
- B6: Chọn làm phần tử giải luôn (gán vào biến pivot_element) và gán row_pivot_element = row
- B7: Liệt kê các phần tử có giá trị bằng với giá trị tuyệt đối của pivot_element vào mảng index_temp
- B8: Gán lại index_temp là phần tử đầu tiên trong mảng, chuyển tới bước 10
- B9: Nếu max_row > pivot_element thì gán pivot_element = max_row, chuyển sang bước 7
- B10: Nếu pivot_element $\neq$ 0 thì thêm row_pivot_element vào mảng index_row, index_temp vào mảng index_column

In [7]:
def find_pivot_element():    
    global index_row, index_column
    index_temp = []
    pivot_element = 0
    for row in range(0, len(matrix)):
        if row in index_row:
            continue  # Bỏ qua vì hàng này đã có phần tử giải
        max_row = np.amax(abs(matrix[row, 0:(len(matrix[0]) - 1)]))  # Tìm phần tử lớn nhất trong hàng row
        if (1 in matrix[row, 0:(len(matrix[0]) - 1)]) or (-1 in matrix[row, 0:(len(matrix[0]) - 1)]):  # Nếu có 1 hoặc -1 trong hàng row => chọn làm phần tử giải
            pivot_element = 1
            row_pivot_element = row
            index_temp = np.where(abs(matrix[row, 0:(len(matrix[0]) - 1)]) == pivot_element)
            index_temp = index_temp[:1]
            index_temp = index_temp[0][0]
            break
        elif max_row > pivot_element:  # Lưu giá trị phần tử giải, tìm vị trí trên ma trận
            pivot_element = max_row
            row_pivot_element = row
            index_temp = np.where(abs(matrix[row, 0:(len(matrix[0]) - 1)]) == pivot_element)
            index_temp = index_temp[:1]
            index_temp = index_temp[0][0]
    if pivot_element != 0:  # Lưu vị trí hàng và cột của phần tử giải
        index_row.append(row_pivot_element)
        index_column.append(int(index_temp))
#         print("Phan tu giai: ", round(matrix[index_row[-1]][index_column[-1]], 10))
#         print("Vi tri: ", index_row[-1] + 1, index_column[-1] + 1)
#         print()

### <font color=RoyalBlue>Khởi tạo hàm biến đổi ma trận theo pp Gauss Jordan <font>
<br>Thuật toán:<br/>
- B1: Thực hiện gói tìm phần tử giải
- B2: Tạo ma trận không có kích thước đúng bằng ma trận A
- B3: Duyệt các dòng, mỗi dòng thực hiện các bước đằng sau
- B4: Gán m là $-\frac{\alpha_{iq}}{\alpha_{pq}}$
- B5: Nhân vào với $\alpha_{pj}$ và lưu vào ma trận không
- B6: Sau khi lưu hết thì cộng ma trận không với ma trận gốc

In [8]:
def Gauss_Jordan_method():
    global matrix
    find_pivot_element()
    zeros_array = np.zeros((len(matrix), len(matrix[0])))  # Tạo 1 ma trận không
    for row in range(0, len(matrix)):
        if row == index_row[-1]:
            continue
        m = - matrix[row][index_column[-1]] / matrix[index_row[-1]][index_column[-1]]  # Tìm m
        zeros_array[row] = matrix[index_row[-1]] * m
#         print(zeros_array)
#         print('======================')
    matrix = matrix + zeros_array

### <font color=RoyalBlue>Hàm chuẩn hóa hệ số của phần tử giải <font>
Để chia hệ số của phần tử giải $=1$

In [10]:
def normalize_pivot_element():
    for i in range(len(index_row)):
        matrix[index_row[i]] = matrix[index_row[i]] / matrix[index_row[i]][index_column[i]]

### <font color=RoyalBlue>Hàm kiểm tra hạng của ma trận <font>
Để có thể đưa ra nhận xét hệ pt có nghiệm duy nhất, vô số nghiệm hay vô nghiệm
<br>Thuật toán:<br/>
- B1: Tạo 2 biến rank1 (Hạng của ma trận hệ số A), rank2 (Hạng của ma trận mở rộng)
- B2: Duyệt từng dòng trong ma trận hệ số A, với mỗi hàng, trị tuyệt đối lớn nhất trong hàng lớn hơn 0 thì rank1 ++
- B3: Áp dụng tương tự B2 cho rank2
- B4: Nếu $rank1 < rank2$ thì hệ vô nghiệm
- B5: Nếu $rank1 >$ kích thước của matrix thì hệ có vô số nghiệm
- B6: Nếu ko phải cả 2 th trên thì hệ có nghiệm duy nhất

In [11]:
def rank():
    rank1 = 0  # Hạng của ma trận hệ số A
    rank2 = 0  # Hạng của ma trận mở rộng
    for row in range(0, len(matrix)):
        if np.amax(abs(matrix[row, 0:(len(matrix[0]) - 1)])) > 0:
            rank1 = rank1 + 1
        if np.amax(abs(matrix[row, 0:len(matrix[0])])) > 0:
            rank2 = rank2 + 1
    if rank1 < rank2:
        print("He PT vo nghiem!")
        f=open("GJ_output.txt","w")
        f.write("He PT vo nghiem!")
        f.close()
    elif rank1 < (len(matrix[0]) - 1):
        print("He PT co vo so nghiem!")
        display_solutions()
    else:
        print("He PT co nghiem duy nhat!")
        display_solutions()
        

### <font color=RoyalBlue>Hàm in kết quả <font>

In [12]:
def display_solutions():
        
    global result
    for column in range(len(matrix[0]) - 1):
        if column in index_column:
            vt = index_column.index(column)
            result[column][0] = matrix[index_row[vt]][len(matrix[0]) - 1]
            for i in range(len(matrix[0]) - 1):
                if i not in index_column:
                    result[column][i + 1] = -matrix[index_row[vt]][i]
        else:
            result[column][column + 1] = 1

    # In ma trận result ra màn hình
    print(result)

    # Xuất kết quả ra file output.txt
    np.savetxt('GJ_output.txt', result, fmt='%.5f')  # %.5f: lấy 5 chữ số sau dấu phẩy ghi vào file


### <font color=RoyalBlue>Chương trình chính <font>

In [13]:
for i in range(0, min(len(matrix), len(matrix[0]))):
    Gauss_Jordan_method()
normalize_pivot_element()
rank()

He PT co vo so nghiem!
[[ 4.  0. -2.]
 [ 0.  0.  1.]]
